In [5]:
import pandas as pd
import numpy as np
import sqlite3
import os

In [6]:
# Connect to DataDict
conn_dd = sqlite3.connect('C:/Users/yutang.xiong/Desktop/IFsHistSeries 8.38 20250207/DataDict.db')
cursor_dd = conn_dd.cursor()
query="SELECT * FROM [DataDict]"
dt_dict=pd.io.sql.read_sql(query,conn_dd)
# close connection to DataDict
conn_dd.close()
dt_dict_core = dt_dict[dt_dict["UsedInPreprocessor"]==True][["Table", "Group", "SubGroup","Definition",'Source', 'Last IFs Update', 'UsedInPreprocessorFileName']].copy()
dt_dict_core.head(3)

,Table,Group,SubGroup,Definition,Source,Last IFs Update,UsedInPreprocessorFileName
37,SeriesLiteracyAdultTotal%,"SociaPolitical, Knowledge, Education",Human Needs,Adult literacy rate is the percentage of peopl...,World Bank,2021/05/18,POPULATION
63,SeriesMilExpendStock,FBIC,None,"Military Expenditures Stock, 10-yr sum, 10%-po...",GIDE,2022/06/10,ECONOMY
202,SeriesPovLineNationalWB2017,SocioPolitical,Equity,"Poverty, income/consumption threshold per per...",World Bank Working Paper No 7606 (Jolliffe and...,2021/07/21,ECONOMY


In [26]:
y_start = 2017
y_end = 2023
# Connect to IFsHistSeries
conn_ifs = sqlite3.connect('C:/Users/yutang.xiong/Desktop/IFsHistSeries 8.38 20250207/IFsHistSeries.db')
cursor_ifs = conn_ifs.cursor()
# get all table names
cursor_ifs.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor_ifs.fetchall()
tables_IFs = [table[0] for table in tables]
#
ifshist_list = []
tb_error = []
for tb in dt_dict_core.Table:
    query=f"SELECT * FROM [{tb}]"  # Or query="SELECT * FROM ["+ table_name +"]"
    dt_ifshist=pd.io.sql.read_sql(query,conn_ifs)
    try:
        columns_y =  [y for y in dt_ifshist.columns[2:-2] if int(y)>=y_start and int(y)<=y_end]  
    except:
        tb_error.append(tb)
        continue
    dt_ifshist = dt_ifshist[[*["Country"],* columns_y]]
    for y in [str(y) for y in range(y_start,y_end+1)]:
        if y not in dt_ifshist.columns:
            dt_ifshist[y] = np.nan
    dt_ifshist.insert(0, "Table", tb)
    ifshist_list.append(dt_ifshist)
#
dt_ifs_core = pd.concat(ifshist_list)
# close connection to IFsHistSeries 
conn_ifs.close()

C:\Users\yutang.xiong\AppData\Local\Temp\ipykernel_17796\1560455910.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dt_ifs_core = pd.concat(ifshist_list)


In [27]:
dt_cover = pd.merge(left = dt_dict_core, right = dt_ifs_core, on = ["Table"], how="right")
for y in dt_ifs_core.columns[2:]:
    print(y, dt_cover[y].notna().sum())

2017 73234
2018 70302
2019 67050
2020 64389
2021 57987
2022 27966
2023 13924


In [21]:
# sum = 356     356 - 154 - 51 - 50 - 8 = 93
dt_cover[(dt_cover["2021"].notna())&(dt_cover["2022"].isna())].drop_duplicates(subset=["Table"]).Source.value_counts()

Source
FAO BATCH PULL                                                                                                                                                                                                  160
World Development Indicators (WDI), World Bank                                                                                                                                                                   62
UNESCO Institute for Statistics (UIS)                                                                                                                                                                            39
BGR; "Reserves, Resources and Availability of Energy Resources"Bundesanstalt für Geowissenschaften und Rohstoffe (BGR) in Hannover; Annual Report. Reserves, Resources and Availability of Energy Resources"     11
FAO FishstatJ software, Global  Aquaculture Production Quantity data                                                                             

In [10]:
# dt_cover[(dt_cover["2017"].notna())&(dt_cover["2018"].isna()) & (dt_cover.Source.str.contains("FAO"))].drop_duplicates(subset=["Table"])["Last IFs Update"].unique()
# dt_cover[(dt_cover["2017"].notna())&(dt_cover["2018"].isna()) & (dt_cover.Source.str.contains("WDI"))].drop_duplicates(subset=["Table"])["Last IFs Update"].unique()
# dt_cover[(dt_cover["2017"].notna())&(dt_cover["2018"].isna()) & (dt_cover.Source.str.contains("UIS"))].drop_duplicates(subset=["Table"])["Last IFs Update"].unique()

In [14]:
# #dt_cover[(dt_cover["2017"].notna())&(dt_cover["2018"].isna()) & (dt_cover.Source.str.contains("WDI"))].drop_duplicates(subset=["Table"])
# dt_uis = dt_cover[(dt_cover["2017"].notna())&(dt_cover["2018"].isna()) & (dt_cover.Source.str.contains("UIS"))].drop_duplicates(subset=["Table"])
# dt_uis = dt_uis[["Table", "Group", "SubGroup", "Definition", "Source", "Last IFs Update"]]
# dt_dict[dt_dict.Table.isin(dt_uis.Table.unique())].to_excel("UIS Preprocessors search 20220223.xlsx", index=False)